<a href="https://colab.research.google.com/github/vkekd25/cp1_preprocessing/blob/main/preprocessing_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 전처리 필요 항목
- 당근마켓 : manner degree, chat 지우기
- title : "구해요", "삽니다" , "구매합니다", "스트랩" , "밴드", "충전", "루프", "스트렙" 제거
- price : 쉼표 제거
- category : 
    - 당근마켓 데이터 : 디지털 기기 빼고 제거
    - 번개장터 데이터 : 웨어러블(워치) 빼고 제거
- location : 당근마켓 데이터랑 지역 단위 맞추기, "전국" ➡ nan 값으로 바꾸기
- article
    - "\n" 제거
    - "미사용", "미개봉", "새 세품", "미사용", "새상품" ➡ 미개봉 column 만들어서 1 값 부여
- 제품별 정가 (판매가) 컬럼 추가
- 할인율 컬럼 추가

# 전처리

## 데이터 불러오기
- web_types : 0=당근마켓, 1=번개장터

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/CodeStates Project 1/데이터/concat_df.csv', index_col = 0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28605 entries, 0 to 8854
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      28605 non-null  object 
 1   price      28605 non-null  object 
 2   like       28605 non-null  int64  
 3   view       28605 non-null  float64
 4   category   28605 non-null  object 
 5   article    28605 non-null  object 
 6   location   28605 non-null  object 
 7   time       28605 non-null  object 
 8   web_types  28605 non-null  int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 2.2+ MB


In [ ]:
df.head(2)

,title,price,like,view,category,article,location,time,web_types
0,애플워치se 44mm GPS 판매 풀박 스트렙증정 애플워치,310000,0,25.0,디지털기기,\n풀박입니다.\n사진 상태 보심 알겠지만 상태 매우 좋아요\n추가 스트랩+케이스 ...,경남,0.024,0
1,애플워치7 41 wifi 미드나이트,370000,0,21.0,디지털기기,\n작년에 구매했고 잘 사용안해서 판매합니다.\n배터리 성능 99%입니다.\n스트랩...,경기도,0.014,0


## 1) price 쉼표 제거 및 형변환
- price : 쉼표 제거하고 int형으로 형변환
- view : float ➡ int형으로 형변환
- time : object ➡ float형으로 형변환

In [ ]:
from numpy import NaN

# price 쉼표 제거 & 형변환

df['view'] = df['view'].astype('int64')
df['time'] = df['time'].replace('?', NaN).astype('float64')
df['price'] = df['price'].str.replace(',', '').astype('int64')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28605 entries, 0 to 8854
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      28605 non-null  object 
 1   price      28605 non-null  int64  
 2   like       28605 non-null  int64  
 3   view       28605 non-null  int64  
 4   category   28605 non-null  object 
 5   article    28605 non-null  object 
 6   location   28605 non-null  object 
 7   time       19750 non-null  float64
 8   web_types  28605 non-null  int64  
dtypes: float64(1), int64(4), object(4)
memory usage: 2.2+ MB


## 2) location 단위 맞추기
- ex) 서울시 강남구 ➡ 서울

In [ ]:
# 수정 전 location
df.tail(10)

,title,price,like,view,category,article,location,time,web_types
8845,갤럭시워치4 핑크골드40mm,250000,1,86,웨어러블(워치),구매후 두번착용 ..시계 안차서 판매합니다.\n구성품 다 있으며 .액정보호필름 부착...,서울특별시 송파구 삼전동,NaN,1
8846,"갤럭시 워치 4 40,44 블투 LET 미개봉 팜이다",200000,1,34,스마트폰,미개봉품 입니다\n연락주세요,전국,NaN,1
8847,갤럭시워치3(41mm),120000,1,47,웨어러블(워치),한달정도 사용한 갤워치3 판매합니다!\n베젤링이 부착돤 상태입니다\n(물론 탈부착은...,광주광역시 남구 효덕동,NaN,1
8848,갤럭시워치 LG U+ SM-R805N,60000,3,103,웨어러블(워치),2019년에 삿어요\n중전기하고 충전독 2개 드립니다.\n화면은 살때 필름 그대로 ...,전국,NaN,1
8849,"갤럭시 워치4 클래식 46mm 블랙 팝니다. (블루투스, 미착용)",295000,2,234,웨어러블(워치),"갤럭시 워치4 클래식 46mm 블랙\nSM-R890 (블루투스, wifi, GPS...",대구광역시 중구 대봉1동,NaN,1
8850,갤럭시워치41mm시계,120000,0,38,기타(시계),에누리 가능하구요 스트랩 드립니다\n,전국,NaN,1
8851,미개봉 갤럭시 워치 4 40mm 실버,220000,2,81,케이스/보호필름/액세서리,미개봉 새상품입니다\n택배 발송은 3월 14일 이후에 가능해요~~,광주광역시 서구 농성1동,NaN,1
8852,삼성 갤럭시 워치 액티브2 44 스테인리스,110000,1,180,웨어러블(워치),제목 그대로 입니다!! 줄 바꾸는 툴킷이 안보이네요 ㅠㅠ 그리고 좀 뻑뻑해영,경상남도 창원시 성산구 웅남동,NaN,1
8853,갤럭시 워치4 블랙 40mm,230000,0,88,웨어러블(워치),제조연월: 2021. 08.\nBluetooth_Wi-Fi_GPS\n\n갤럭시 워치...,경기도 화성시 팔탄면,NaN,1
8854,갤럭시워치 액티브2 풀박스,90000,1,107,웨어러블(워치),구성품:본체+충전기+박스+스트랩+강화필름\n작년에 구입하였습니다.,경기도 평택시 서정동,NaN,1


In [ ]:
# location 단위 맞추는 함수
def lo(df):
    if '충청남도' in df:
        df = '충남'
    elif '충청북도' in df:
        df =  '충북'
    elif '경상남도' in df:
        df = '경남'
    elif '경상북도' in df:
        df ='경북'
    elif '전라남도' in df:
        df = '전남'
    elif '전라북도' in df:
        df ='전북'
    elif '강원도' in df:
        df = '강원도'
    elif '경기도' in df:
        df = '경기도'
    else:
        df = df[:2]
    return df

In [ ]:
# 함수 적용
df['location'] = df['location'].apply(lo)

In [ ]:
# 이상값 확인 -> 251개 row 제거
df['location'].value_counts()

경기도    7012
서울     6257
전국     2571
인천     1668
부산     1501
대구     1309
경남     1221
경북      887
충남      855
대전      822
광주      786
충북      697
전북      668
울산      620
강원도     542
전남      516
제주      226
세종      200
0        79
1        62
2        25
5        13
4        12
7         7
3         5
10        4
6         4
9         4
11        4
8         3
19        2
32        2
15        2
42        2
5,        2
17        2
애플        1
31        1
13        1
12        1
25        1
23        1
60        1
26        1
7,        1
64        1
00        1
35        1
27        1
Name: location, dtype: int64

In [ ]:
# 이상값 제거
df = df[df["location"].str.contains("경기도|서울|전국|인천|부산|대구|경남|경북|충남|대전|광주|충북|전북|울산|강원도|전남|제주|세종")]

In [ ]:
# 제거 후 location values
df['location'].value_counts()

경기도    7012
서울     6257
전국     2571
인천     1668
부산     1501
대구     1309
경남     1221
경북      887
충남      855
대전      822
광주      786
충북      697
전북      668
울산      620
강원도     542
전남      516
제주      226
세종      200
Name: location, dtype: int64

In [ ]:
# 제거 후 shape
df.shape

(28358, 9)

## 3) article에서 미개봉 컬럼 추출
- "미사용", "미개봉", "새 세품", "미사용", "새상품" ➡ 미개봉 column 만들어서 1 값 부여

In [ ]:
import copy

df['unused'] = df['article'].str.contains("미개봉 | 미사용 | 미착용 | 미개봉 | 새제품 | 새 제품 | 새상품 |새 상품| 미착용").apply(lambda x : 1 if x==True else 0)

In [ ]:
# 미개봉 제품 3732개 (전체의 13%)
df[df['unused']==1]

,title,price,like,view,category,article,location,time,web_types,unused
7,애플워치7 스타라이트 GPS 41mm,399000,0,23,디지털기기,\n이미 44를 사용중이라 보관 해두고있는 41은 판매하려고합니다 :) \n2022...,대구,0.019,0,1
8,애플워치 40mm 스트랩,6000,0,17,디지털기기,\n거의 새상품 입니다. 엄청 가벼워요’!\n,경북,0.020,0,1
9,애플워치se 44mm 미개봉 삽니당,300000,0,1,삽니다,\n워치 미개봉 삽니다 연락부탁드려요\n,충북,0.021,0,1
35,애플워치5 스테인리스 블랙 44미리 미개봉 팝니다,400000,5,266,디지털기기,\n애플워치5 스테인리스 블랙 44미리 미개봉 팝니다\nSE동일칩셋 + 셀룰러 + ...,서울,0.030,0,1
38,애플워치6 실버 40mm 스테인리스 판매합니다.,465000,0,9,디지털기기,\n애플워치6 실버 40mm 스테인리스 판매합니다.\n사진처럼 보증기간은 23년 3...,서울,0.022,0,1
...,...,...,...,...,...,...,...,...,...,...
8829,갤럭시워치3 블루투스 45mm A급 제품,140000,1,365,웨어러블(워치),갤럭시워치3 블루투스 45mm A급 제품\n\n구성품: 사진상본체+충전기\n\n커버...,대구,NaN,1,1
8837,미개봉 새제품] 갤럭시 워치4 클래식 46 LTE,340000,0,64,생활가전,[미개봉 새제품] 갤럭시 워치4 클래식 46 LTE \n실버색상 자급제\n\n블랙보...,서울,NaN,1,1
8844,갤럭시워치 4 46mm,300000,1,54,웨어러블(워치),구매하고 사용하지 않아 판매합니다 \n미개봉 상태고요 편하게 연락주시면 감사드리겠습니다,전국,NaN,1,1
8849,"갤럭시 워치4 클래식 46mm 블랙 팝니다. (블루투스, 미착용)",295000,2,234,웨어러블(워치),"갤럭시 워치4 클래식 46mm 블랙\nSM-R890 (블루투스, wifi, GPS...",대구,NaN,1,1


## 4) 애플워치와 갤럭시워치 모두 포함하는 row 제거

In [ ]:
# 애플워치와 갤럭시워치 모두 포함하는 row 제거
# 28358 -> 28179

df = df[(df['title'].str.contains('애플')) & (df['title'].str.contains('갤럭시')) == False]
df.shape

(28179, 10)

### 단어 제거

In [ ]:
# 밴드나 스트랩이 포함된 단어 제거
df = df[df["title"].str.contains("스트랩|밴드|충전|루프|스트렙|박스|케이스|필름|거치대|시계줄|큐빅줄|쿠폰|보호|사기") == False]

In [ ]:
# 제거 후 약 28000개 -> 18761개만 남음
df.shape

(18761, 10)

In [ ]:
# 어텐션 적용
df['attention']= 0.7*df['like'].copy(deep=True) + 0.3*df['view'].copy(deep=True)
df["like/views(%)"]=100*df['like'].copy(deep=True) /df['view'].copy(deep=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# 판매가 아닌 "구매" row 제거
# 제거 후 18870개 -> 18014개
df = df[df["title"].str.contains("삽니다|구매|원합니다|구해|교환|교신|구합|바꾸|사요|매입")== False]
df.shape

(17957, 12)

### 모델 분류 - 애플워치

In [ ]:
# 애플워치만 분류하기
df_aw = df[df["title"].str.contains("애플|Apple|apple")] 
df_aw.shape

(9573, 12)

In [ ]:
# GPS /celluar 분류하기

# GPS 분류하기
df_aw_gps = df_aw[df_aw["title"].str.contains("셀룰러|cellular")== False].copy(deep=True) 
df_aw_gps["gps/cell"] = "gps"

# Celluar 분류하기
df_aw_cellular = df_aw[df_aw["title"].str.contains("셀룰러|cellular")].copy(deep=True)  
df_aw_cellular["gps/cell"] = "cellular"


df_aw = pd.concat([df_aw_gps,df_aw_cellular])
df_aw


,title,price,like,view,category,article,location,time,web_types,unused,attention,like/views(%),gps/cell
1,애플워치7 41 wifi 미드나이트,370000,0,21,디지털기기,\n작년에 구매했고 잘 사용안해서 판매합니다.\n배터리 성능 99%입니다.\n스트랩...,경기도,0.014,0,0,6.3,0.000000,gps
2,애플워치,130000,1,78,디지털기기,\n애플워치로삽니당\n,서울,0.014,0,0,24.1,1.282051,gps
4,애플워치se 40mm 팝니당,260000,1,15,디지털기기,"\n착용 4회했고 갤럭시로 바꾸면서 팔게되었습니다\n생활기스, 찍힘 등 파손부위 X...",경기도,0.016,0,0,5.2,6.666667,gps
6,애플워치5 44 스테인리스 실버,400000,0,77,디지털기기,\n깨끗한 s급 상품입니다\n박스없이 본체+충전선+폼폼푸린루프(지금 껴진 노란뚱땡이...,서울,0.018,0,0,23.1,0.000000,gps
7,애플워치7 스타라이트 GPS 41mm,399000,0,23,디지털기기,\n이미 44를 사용중이라 보관 해두고있는 41은 판매하려고합니다 :) \n2022...,대구,0.019,0,1,6.9,0.000000,gps
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6856,애플워치6/ 44mm / 셀룰러+gps 실버 판매합니다,335000,31,1261,웨어러블(워치),구입일자:2021.10.16일\n보증기간:2022.10.15일\n케이스 쉬운상태로 ...,전국,NaN,1,0,400.0,2.458366,cellular
6872,애플워치6 44mm 로즈골드 셀룰러 풀박 팝니다!,400000,1,78,웨어러블(워치),애플워치6 44mm 로즈골드 풀박 셀룰러+gps 팝니다! 풀박이에요! \n선물 받아...,부산,NaN,1,0,24.1,1.282051,cellular
6888,애플워치6 스테인리스 그래파이트40 셀룰러,430000,5,333,웨어러블(워치),2020.12월 구매\n2022.12월 까지 애플케어 \n40mm입니다\n배터리 성...,경기도,NaN,1,0,103.4,1.501502,cellular
6963,애플워치 se 40mm gps+셀룰러 실버 미개봉,370000,30,540,웨어러블(워치),제조년 2022년입니당,대전,NaN,1,0,183.0,5.555556,cellular


In [ ]:
# 에르메스 /일반 분류하기

# 에르메스인 항목 가져오기
df_aw_hermes = df_aw[df_aw["title"].str.contains("에르|hermes|헤르")].copy(deep=True)  
df_aw_hermes["edition"] = "에르메스"

# 에르메스 아닌 항목들 분류하기 (에르메스가 아닌 모든걸 일반으로 분류)
df_aw_nhermes = df_aw[df_aw["title"].str.contains("에르|hermes|헤르")== False].copy(deep=True) 
df_aw_nhermes["edition"] = "일반"


df_aw = pd.concat([df_aw_hermes,df_aw_nhermes])
df_aw

,title,price,like,view,category,article,location,time,web_types,unused,attention,like/views(%),gps/cell,edition
242,애플워치7 에르메스 미개봉 이번주만 급처 합니다.,1250000,0,73,디지털기기,\n7월 구매후설물받은것도저히 필요 없어서 급처 합니다.\n애케플기간다남아있구요\n...,충북,0.041667,0,1,21.9,0.000000,gps,에르메스
243,(일괄판매)애플워치7 에르메스 45mm 누아르 애케플,2000000,1,75,디지털기기,\n일괄구매 200만원\n워치7 에르메스 누아르 100만\n누아 싱글투어 (사용감 ...,서울,0.041667,0,0,23.2,1.333333,gps,에르메스
302,애플워치 7 에르메스 누아르 45mm 판매합니다,1350000,1,234,디지털기기,\n작년 12월 구매 1월에 애플케어플러스 가입했습니다 \n25.1.26 까지입니다...,서울,0.083333,0,0,70.9,0.427350,gps,에르메스
354,애플워치6 에르메스 싱글투어 44m,900000,0,127,디지털기기,"\n애플워치6 실버 스테인리스 스틸 케이스, 그리고 아뜰라주 싱글 투어 블랙 44M...",서울,0.125000,0,0,38.1,0.000000,gps,에르메스
452,애플워치7 에르메스 에벤느 바레니아 가죽싱글투어 디플로이먼트 버클 판매(줄만판매),405000,2,279,디지털기기,\n애플워치7나올때 구매했습니다. \n몃번밖에 사용했으나 가죽 특성상 주름있어요. ...,대전,0.125000,0,0,85.1,0.716846,gps,에르메스
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6856,애플워치6/ 44mm / 셀룰러+gps 실버 판매합니다,335000,31,1261,웨어러블(워치),구입일자:2021.10.16일\n보증기간:2022.10.15일\n케이스 쉬운상태로 ...,전국,NaN,1,0,400.0,2.458366,cellular,일반
6872,애플워치6 44mm 로즈골드 셀룰러 풀박 팝니다!,400000,1,78,웨어러블(워치),애플워치6 44mm 로즈골드 풀박 셀룰러+gps 팝니다! 풀박이에요! \n선물 받아...,부산,NaN,1,0,24.1,1.282051,cellular,일반
6888,애플워치6 스테인리스 그래파이트40 셀룰러,430000,5,333,웨어러블(워치),2020.12월 구매\n2022.12월 까지 애플케어 \n40mm입니다\n배터리 성...,경기도,NaN,1,0,103.4,1.501502,cellular,일반
6963,애플워치 se 40mm gps+셀룰러 실버 미개봉,370000,30,540,웨어러블(워치),제조년 2022년입니당,대전,NaN,1,0,183.0,5.555556,cellular,일반


In [ ]:
# 알루미늄 / 스테인리스 / 티타늄 분류하기

# 스테인리스 분류하기
df_aw_stainless = df_aw[df_aw["title"].str.contains("스테인|스태인|스텐|stain|Stain|스뎅")].copy(deep=True) 
df_aw_stainless["material"] = "stainless"

# 티타늄 분류하기
df_aw_titanium = df_aw[df_aw["title"].str.contains("티타늄|titanium|Titanium")].copy(deep=True)  
df_aw_titanium["material"] = "titanium"

# 나머지는 알루미늄으로 분류
df_aw_aluminum = df_aw[df_aw["title"].str.contains("스테인|스태인|스텐|stain|Stain|스뎅|티타늄|titanium|Titanium")== False].copy(deep=True) 
df_aw_aluminum["material"] = "aluminum"

df_aw = pd.concat([df_aw_stainless,df_aw_titanium,df_aw_aluminum])
df_aw

,title,price,like,view,category,article,location,time,web_types,unused,attention,like/views(%),gps/cell,edition,material
663,애플워치7 45mm 실버 스테인리스 에르메스 에디션 애플케어플러스,1300000,2,417,디지털기기,\n1.제품:애플워치7 45mm 실버 스테인리스 애플케어플러스 24.12.18까지\...,서울,0.208333,0,0,126.5,0.479616,gps,에르메스,stainless
697,애플워치7 45mm 실버 스테인리스 에르메스 에디션 애플케어플러스,1300000,3,181,디지털기기,\n1.제품:애플워치7 45mm 실버 스테인리스 애플케어플러스 24.12.18까지\...,서울,0.208333,0,0,56.4,1.657459,gps,에르메스,stainless
700,애플워치7 45mm 실버 스테인리스 에르메스 에디션 애플케어플러스,1300000,3,181,디지털기기,\n1.제품:애플워치7 45mm 실버 스테인리스 애플케어플러스 24.12.18까지\...,서울,0.208333,0,0,56.4,1.657459,gps,에르메스,stainless
1228,에르메스 애플워치 7세대 41mm 스테인리스 스틸 골드 싱글투어,1230000,5,442,디지털기기,\n에르메스 애플워치 7세대 41mm 스테인리스 스틸 골드 싱글투어 판매합니다\n2...,서울,0.375000,0,0,136.1,1.131222,gps,에르메스,stainless
1252,"애플워치 7 45mm 스테인리스 GPS+Cellular, 에르메스 에디션",1250000,3,356,디지털기기,"\n- 제품 : Apple Watch Hermes GPS+Cellular,45mm ...",경기도,0.375000,0,0,108.9,0.842697,gps,에르메스,stainless
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6777,애플워치 se nike 44mm 셀룰러 판매,270000,11,232,웨어러블(워치),애플워치 se nike 44 스페이스그레이 셀룰러 모델입니다\n배터리 성능 95퍼\...,충남,NaN,1,0,77.3,4.741379,cellular,일반,aluminum
6841,애플워치se 44mm 셀룰러,250000,1,195,웨어러블(워치),뒷면 티안나게약파손이거든여 착용하면 티안나요 방수문제없는게싹테스트했어요 구매하시고 ...,전국,NaN,1,0,59.2,0.512821,cellular,일반,aluminum
6856,애플워치6/ 44mm / 셀룰러+gps 실버 판매합니다,335000,31,1261,웨어러블(워치),구입일자:2021.10.16일\n보증기간:2022.10.15일\n케이스 쉬운상태로 ...,전국,NaN,1,0,400.0,2.458366,cellular,일반,aluminum
6872,애플워치6 44mm 로즈골드 셀룰러 풀박 팝니다!,400000,1,78,웨어러블(워치),애플워치6 44mm 로즈골드 풀박 셀룰러+gps 팝니다! 풀박이에요! \n선물 받아...,부산,NaN,1,0,24.1,1.282051,cellular,일반,aluminum


In [ ]:
df_aw['material'].value_counts()

aluminum     9096
stainless     445
titanium       32
Name: material, dtype: int64

In [ ]:
# 애플워치-세부모델 별로 분류

df_aw_7 = df_aw[df_aw["title"].str.contains("치7|치 7")]    #38
df_aw_7_41 = df_aw_7[df_aw_7["title"].str.contains("41")].copy(deep=True)  #17
df_aw_7_41["model"] = "애플워치"
df_aw_7_41["series"] = "7"
df_aw_7_41["size"] = "41mm"
df_aw_7_45 = df_aw_7[df_aw_7["title"].str.contains("45")].copy(deep=True)  #15
df_aw_7_45["model"] = "애플워치"
df_aw_7_45["series"] = "7"
df_aw_7_45["size"] = "45mm"

df_aw_se = df_aw[df_aw["title"].str.contains("치se|치 se|치SE|치 SE")]  #33
df_aw_se_40 = df_aw_se[df_aw_se["title"].str.contains("40")].copy(deep=True) 
df_aw_se_40["model"] = "애플워치"
df_aw_se_40["series"] = "se"
df_aw_se_40["size"] = "40mm"
df_aw_se_44 = df_aw_se[df_aw_se["title"].str.contains("44")].copy(deep=True) 
df_aw_se_44["model"] = "애플워치"
df_aw_se_44["series"] = "se"
df_aw_se_44["size"] = "44mm"

df_aw_6 = df_aw[df_aw["title"].str.contains("치6|치 6")]  #33
df_aw_6_40 = df_aw_6[df_aw_6["title"].str.contains("40")].copy(deep=True) 
df_aw_6_40["model"] = "애플워치"
df_aw_6_40["series"] = "6"
df_aw_6_40["size"] = "40mm"
df_aw_6_44 = df_aw_6[df_aw_6["title"].str.contains("44")].copy(deep=True) 
df_aw_6_44["model"] = "애플워치"
df_aw_6_44["series"] = "6"
df_aw_6_44["size"] = "44mm"

df_aw_5 = df_aw[df_aw["title"].str.contains("치5|치 5")] #20
df_aw_5_40 =df_aw_5[df_aw_5["title"].str.contains("40")].copy(deep=True)
df_aw_5_40["model"] = "애플워치"
df_aw_5_40["series"] = "5"
df_aw_5_40["size"] = "40mm"
df_aw_5_44 = df_aw_5[df_aw_5["title"].str.contains("44")].copy(deep=True)
df_aw_5_44["model"] = "애플워치"     
df_aw_5_44["series"] = "5"
df_aw_5_44["size"] = "44mm"       
             
df_aw_4 = df_aw[df_aw["title"].str.contains("치4|치 4")] #20
df_aw_4_40 =df_aw_4[df_aw_4["title"].str.contains("40")].copy(deep=True)
df_aw_4_40["model"] = "애플워치"
df_aw_4_40["series"] = "4"
df_aw_4_40["size"] = "40mm"
df_aw_4_44 = df_aw_4[df_aw_4["title"].str.contains("44")].copy(deep=True)
df_aw_4_44["model"] = "애플워치"
df_aw_4_44["series"] = "4"
df_aw_4_44["size"] = "44mm"  
                      
df_aw_3 = df_aw[df_aw["title"].str.contains("치3|치 3")] #30
df_aw_3_38 = df_aw_3[df_aw_3["title"].str.contains("38")].copy(deep=True)
df_aw_3_38["model"] = "애플워치"
df_aw_3_38["series"] = "3"
df_aw_3_38["size"] = "38mm"
df_aw_3_42 = df_aw_3[df_aw_3["title"].str.contains("42")].copy(deep=True)
df_aw_3_42["model"] = "애플워치"
df_aw_3_42["series"] = "3"
df_aw_3_42["size"] = "42mm"

df_aw_2 = df_aw[df_aw["title"].str.contains("치2|치 2")].copy(deep=True) #2
df_aw_2_38 = df_aw_2[df_aw_2["title"].str.contains("38")].copy(deep=True)
df_aw_2_38["model"] = "애플워치"
df_aw_2_38["series"] = "2"
df_aw_2_38["size"] = "38mm"
df_aw_2_42 = df_aw_2[df_aw_2["title"].str.contains("38")].copy(deep=True)
df_aw_2_42["model"] = "애플워치"
df_aw_2_42["series"] = "2"
df_aw_2_42["size"] = "42mm"



df_aw_1 = df_aw[df_aw["title"].str.contains("치1|치 1")].copy(deep=True) #0
df_aw_1_38 = df_aw_1[df_aw_1["title"].str.contains("38")].copy(deep=True)
df_aw_1_38["model"] = "애플워치"
df_aw_1_38["series"] = "1"
df_aw_1_38["size"] = "38mm"
df_aw_1_42 = df_aw_1[df_aw_1["title"].str.contains("38")].copy(deep=True)
df_aw_1_42["model"] = "애플워치"
df_aw_1_42["series"] = "1"
df_aw_1_42["size"] = "42mm"

In [ ]:
# 분류되어 모델별로 나눠진 애플 테이블들을 다시 하나로 합치기

df_apple_new = pd.concat([df_aw_7_41,df_aw_7_45, df_aw_se_40, df_aw_se_44, df_aw_6_40,df_aw_6_44, df_aw_5_40,df_aw_5_44,df_aw_4_40,df_aw_4_44,df_aw_3_38,df_aw_3_42 ,df_aw_2_38,
                          df_aw_2_42, df_aw_1_38, df_aw_1_42])
df_apple_new

,title,price,like,view,category,article,location,time,web_types,unused,attention,like/views(%),gps/cell,edition,material,model,series,size
1228,에르메스 애플워치 7세대 41mm 스테인리스 스틸 골드 싱글투어,1230000,5,442,디지털기기,\n에르메스 애플워치 7세대 41mm 스테인리스 스틸 골드 싱글투어 판매합니다\n2...,서울,0.375000,0,0,136.1,1.131222,gps,에르메스,stainless,애플워치,7,41mm
449,애플워치7 41mm (실버) 스테인리스 팝니다.,650000,0,209,디지털기기,\n애플워치7 41mm (실버) 스테인리스 팝니다.\n충전기 및 풀박스입니다.\n스...,경북,0.125000,0,0,62.7,0.000000,gps,일반,stainless,애플워치,7,41mm
602,애플워치 7 스테인리스 41mm실버 팝니다,600000,14,1164,디지털기기,\n구매후 실착용 10번도 하지 않은 상태 좋은 애플워치 7 스테인리스 팝니다\n배...,충북,0.166667,0,0,359.0,1.202749,gps,일반,stainless,애플워치,7,41mm
1792,애플워치7 41mm 스테인리스 미사용팝니다,650000,3,118,디지털기기,\n처음에 한번켜보고 초기셋팅후 바로 박스에 보관되었던 제품입니다\n본체만입니다 밀...,서울,0.541667,0,0,37.5,2.542373,gps,일반,stainless,애플워치,7,41mm
1804,애플워치7 41mm 스테인리스 실버 미사용팝니다,650000,3,123,디지털기기,\n본체만 있습니다 밀레니즈루프줄은 팔리고 없습니다\n스타라이트 스포츠밴드 하나 가...,서울,0.541667,0,0,39.0,2.439024,gps,일반,stainless,애플워치,7,41mm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4924,애플워치 1세대 38mm,80000,4,147,디지털기기,\n기기 단품과 충전기입니다.\n저번달에 배터리 자가 교체하고 쓰질 않아서 배터리는...,부산,2.000000,0,0,46.9,2.721088,gps,일반,aluminum,애플워치,1,42mm
5167,38mm 애플워치1,50000,5,151,디지털기기,\n애플워치 입문용으로 당근에서 구입했다가 재당근해요! 보시는바와 같이 생활기스 있...,서울,2.000000,0,0,48.8,3.311258,gps,일반,aluminum,애플워치,1,42mm
7776,애플워치 1세대 스포츠 38mm 알루미늄,30000,4,76,디지털기기,\n옛날에 선물받은거 찾아서 잠금풀려다가 락걸렸네요 부품용이나 팔찌로 쓰실분 있으실...,서울,3.000000,0,0,25.6,5.263158,gps,일반,aluminum,애플워치,1,42mm
55,애플워치 1 38mm 1세대 스그,85000,4,48,웨어러블(워치),기스 거의 없고\n작년 초에 배터리 교체인가 액정 교체인가 했었어요\n그 이후로 두...,전국,NaN,1,0,17.2,8.333333,gps,일반,aluminum,애플워치,1,42mm


In [ ]:
# 중복 칼럼 제거
# 최종 shape -> 6712
df_apple_new = df_apple_new.drop_duplicates()
df_apple_new.shape

(7313, 18)

### 모델 분류 - 갤럭시워치

In [ ]:
# 갤럭시워치만 분류하기
df_gw = df[df["title"].str.contains("갤럭시")]
df_gw.shape

(8162, 12)

In [ ]:
#GPS /celluar 분류하기

#blooth 분류하기
df_gw_bluetooth = df_gw[~df_gw["title"].str.contains("lte|엘티이|엘티|티이|셀룰러|cell")].copy(deep=True) 
df_gw_bluetooth["gps/cell"] = "bluetooth"

#lte 분류하기
df_gw_lte = df_gw[df_gw["title"].str.contains("lte|엘티이|엘티|티이|셀룰러|cell")].copy(deep=True)  
df_gw_lte["gps/cell"] = "lte"

df_gw = pd.concat([df_gw_bluetooth,df_gw_lte])
df_gw

#골프 에디션x
df_gw_general = df_gw[~df_gw["title"].str.contains("golf|골프|프에|프애|에디션|애디션|edition")].copy(deep=True)  
df_gw_general["edition"] = "general"

#골프 에디션
df_gw_golf = df_gw[df_gw["title"].str.contains("golf|골프|프에|프애|에디션|애디션|edition")].copy(deep=True)  
df_gw_golf["edition"] = "golf"


df_gw = pd.concat([df_gw_general,df_gw_golf])
df_gw.shape

(8162, 14)

In [ ]:
#material
df_gw_aluminium = df_gw[~(df_gw["title"].str.contains("스태|스테|테인|태인|리스|stain|less") | df_gw["title"].str.contains("티타|티탸|타늄|탸늄|타눔|탸눔|tita|tanum"))].copy(deep=True)
df_gw_aluminium['material'] = 'aluminium'

df_gw_stainless = df_gw[df_gw["title"].str.contains("스태|스테|테인|태인|리스|stain|less")].copy(deep=True)
df_gw_stainless['material'] = 'stainless'

df_gw_titanum = df_gw[df_gw["title"].str.contains("티타|티탸|타늄|탸늄|타눔|탸눔|tita|tanum")].copy(deep=True)
df_gw_titanum['material'] = 'titanum'

df_gw = pd.concat([df_gw_aluminium,df_gw_stainless, df_gw_titanum])
df_gw

,title,price,like,view,category,article,location,time,web_types,unused,attention,like/views(%),gps/cell,edition,material
9875,갤럭시 워치5 44mm 그라파이트(블루투스),300000,14,762,디지털기기,\n지난 주 예약판매로 구매한 제품\n아이폰 및 애플워치 사용이 그나마 편해서\n개...,서울,0.015,0,1,238.4,1.837270,bluetooth,general,aluminium
9877,갤럭시 워치4 클래식 42mm,181000,0,333,생활가전,"\n갤럭시 워치4 42mm 두번째사진 생활기스 있습니다\n박스 , 충전기 있구요 ...",경기도,0.016,0,0,99.9,0.000000,bluetooth,general,aluminium
9880,갤럭시워치4-40mm,120000,6,100,디지털기기,\n스트랩부분이 살짝 색이 바랜거말고는 멀쩡해요 상자랑 없어서 저렴하게 내놔요\n다...,전남,0.018,0,0,34.2,6.000000,bluetooth,general,aluminium
9883,갤럭시워치4 40mm 블루투스,90000,5,69,디지털기기,\n약 6개월 사용 깨끗합니다.\n풀구성품 9만원에 날립니다.\n네고X\n,서울,0.018,0,0,24.2,7.246377,bluetooth,general,aluminium
9885,갤럭시워치4클래식 46mm 풀박,190000,7,84,디지털기기,\n22년1월구매\n액정기스없어요\n효성도서관근처서 직거래원합니다\n,인천,0.019,0,0,30.1,8.333333,bluetooth,general,aluminium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18039,갤럭시워치3 티타늄 골프에디션 쿨거네고,190000,1,75,디지털기기,"\n워치5 구매하게되어 판매합니다\n사진에 보이는 것처럼 흠집, 사용감 있습니다\n...",서울,4.000,0,1,23.2,1.333333,bluetooth,golf,titanum
18738,갤럭시워치 5 프로 골프에디션ㆍ45mm블래티타늄,500000,10,983,디지털기기,\n미개봉\n갤럭시워치 5 프로 골프에디션\n 45mmㆍ블래티타늄\n,광주,5.000,0,0,301.9,1.017294,bluetooth,golf,titanum
18841,갤럭시워치3 45미리 티타늄 골프에디션 미개봉 새상품,210000,1,136,스포츠/레저,\n금일 선물 배송 받은 미개봉 새상품입니다.\n아이폰 연동 가능한 제품입니다.\n...,경기도,5.000,0,1,41.5,0.735294,bluetooth,golf,titanum
6285,갤럭시 워치3 티타늄 골프에디션,220000,10,383,웨어러블(워치),미사용 워치 3 티타늄 골프에디션입니다\n사은품은 가죽줄 추가로 드립니다,경기도,NaN,1,0,121.9,2.610966,bluetooth,golf,titanum


In [ ]:
# 갤럭시워치-세부모델 별로 
# 갤럭시 기어 s1(2013년),s2(2014년),s3
# 2018년 갤럭시 워치(기어 s3 이후)시작 - 애플워치 4세대 출시일 비슷
# 갤럭시 워치1 - 42mm/46mm
# 갤럭시 워치2 - 없고
# 갤럭시 워치3 - 41mm/45mm
# 갤럭시 워치4 - 40mm/44mm, 골프 에디션
# 갤럭시 워치4클래식 - 42mm/46mm
# 갤럭시 워치5 - 40mm/44mm, 골프 에디션
# 갤럭시 워치5pro - 45mm, 골프 에디션
# 갤럭시 워치 액티브(2019년) - 40mm   - 액티브는 애플워치 se와 비슷
# 갤럭시 워치 액티브 2 - 40mm / 44mm
# bluetooth/lte, 사이즈, 시리즈, 에디션, 모델명


# 갤럭시워치5
df_gw_5 = df_gw[df_gw["title"].str.contains("시5|시 5|치5|치 5")]
df_gw_5_40 = df_gw_5[df_gw_5["title"].str.contains("40")].copy(deep=True)  #468
df_gw_5_40["model"] = "갤럭시워치"
df_gw_5_40['size'] = '40mm'
df_gw_5_40['series'] = '5'

df_gw_5_44 = df_gw_5[df_gw_5["title"].str.contains("44")].copy(deep=True)  #416
df_gw_5_44["model"] = "갤럭시워치"
df_gw_5_44['size'] = '44mm'
df_gw_5_44['series'] = '5'

# 갤럭시워치5 pro
df_gw_pro = df_gw[df_gw["title"].str.contains("5pro|5 pro|5프로|5 프로|치프로|치 프로|치pro|치 pro", case = False)].copy(deep=True) #499
df_gw_pro["model"] = "갤럭시워치"# 갤럭시 워치프로는 45mm만 있음
df_gw_pro['size'] = '45mm'
df_gw_pro['series'] = '5 pro'

# 갤럭시워치4
df_gw_4 = df_gw[df_gw["title"].str.contains("시4|시 4|치4|치 4")].copy(deep=True)
df_gw_4_40 = df_gw_4[df_gw_4["title"].str.contains("40")].copy(deep=True) # 1065
df_gw_4_40["model"] = "갤럭시워치"
df_gw_4_40['size'] = '40mm'
df_gw_4_40['series'] = '4'

df_gw_4_44 = df_gw_4[df_gw_4["title"].str.contains("44")].copy(deep=True) # 1181
df_gw_4_44["model"] = "갤럭시워치"
df_gw_4_44['size'] = '44mm'
df_gw_4_44['series'] = '5'

# 갤럭시워치4 클래식
df_gw_classic = df_gw[df_gw["title"].str.contains("classic|클래식|치클|치 클|시클|시 클", case = False)].copy(deep=True) 
df_gw_classic_42 = df_gw_classic[df_gw_classic["title"].str.contains("42")].copy(deep=True) # 240
df_gw_classic_42["model"] = "갤럭시워치"
df_gw_classic_42['size'] = '42mm'
df_gw_classic_42['series'] = '4 classic'

df_gw_classic_46 = df_gw_classic[df_gw_classic["title"].str.contains("46")].copy(deep=True) # 438
df_gw_classic_46["model"] = "갤럭시워치"
df_gw_classic_46['size'] = '46mm'
df_gw_classic_46['series'] = '4 clasic'


# 갤럭시워치 3
df_gw_3 = df_gw[df_gw["title"].str.contains("시3|시 3|치3|치 3")].copy(deep=True) 
df_gw_3_41 = df_gw_3[df_gw_3["title"].str.contains("41")].copy(deep=True) # 225
df_gw_3_41["model"] = "갤럭시워치"
df_gw_3_41['size'] = '41mm'
df_gw_3_41['series'] = '3'


df_gw_3_45 = df_gw_3[df_gw_3["title"].str.contains("45")].copy(deep=True) # 545
df_gw_3_45["model"] = "갤럭시워치"
df_gw_3_45['size'] = '45mm'
df_gw_3_45['series'] = '3'

# 갤럭시워치 1
df_gw_1 = df_gw[df_gw["title"].str.contains("시1|시 1|치1|치 1")].copy(deep=True) # 157, + 갤럭시워치 있음
df_gw_1_42 = df_gw_1[df_gw_1["title"].str.contains("42")].copy(deep=True) # 29
df_gw_1_42["model"] = "갤럭시워치"
df_gw_1_42['size'] = '42mm'
df_gw_1_42['series'] = '1'

df_gw_1_46 = df_gw_1[df_gw_1["title"].str.contains("46")].copy(deep=True) # 52
df_gw_1_46["model"] = "갤럭시워치"
df_gw_1_46['size'] = '46mm'
df_gw_1_46['series'] = '1'

# 갤럭시워치 액티브1
df_gw_active = df_gw[df_gw["title"].str.contains("브1|브 1|ive1|ive 1")].copy(deep=True)
df_gw_active_40 = df_gw_active[df_gw_active["title"].str.contains("40")].copy(deep=True) # 6
df_gw_active_40["model"] = "갤럭시워치"
df_gw_active_40['size'] = '40mm'
df_gw_active_40['series'] = 'active1'

# 갤럭시워치 액티브2
df_gw_active_2 = df_gw[df_gw["title"].str.contains("브2|브 2|ve2|ve 2")].copy(deep=True) # 181
df_gw_active_2_40 = df_gw_active_2[df_gw_active_2["title"].str.contains("40")].copy(deep=True)
df_gw_active_2_40["model"] = "갤럭시워치"
df_gw_active_2_40['size'] = '40mm'
df_gw_active_2_40['series'] = 'active2'

df_gw_active_2_44 = df_gw_active_2[df_gw_active_2["title"].str.contains("40")].copy(deep=True)
df_gw_active_2_44["model"] = "갤럭시워치"
df_gw_active_2_44['size'] = '44mm'
df_gw_active_2_44['series'] = 'active2'

df_galaxy_new = pd.concat([df_gw_active_2_44, df_gw_active_2_40, df_gw_active_40, df_gw_1_46, df_gw_1_42, df_gw_3_45, df_gw_3_41, 
                   df_gw_classic_46, df_gw_classic_42, df_gw_4_44, df_gw_4_40, df_gw_pro, df_gw_5_44, df_gw_5_40])
df_galaxy_new

,title,price,like,view,category,article,location,time,web_types,unused,attention,like/views(%),gps/cell,edition,material,model,size,series
10180,갤럭시 워치 액티브2 40mm 정품,150000,0,197,디지털기기,\n2020년도 구매후 1년정도차고 1년동안 방치했습니다. 새로운 주인만나서 사용되...,경기도,0.083333,0,0,59.1,0.000000,bluetooth,general,aluminium,갤럭시워치,44mm,active2
10275,갤럭시워치 액티브 2 40mm,65000,5,407,디지털기기,\n갤럭시워치 바꾸면서 쓰던거 내놓아용ㅎㅎ초기화 시켜놓은 상태이고 정상작동 다 잘됩...,울산,0.125000,0,0,125.6,1.228501,bluetooth,general,aluminium,갤럭시워치,44mm,active2
10276,갤럭시워치액티브2 40mm 판매합니다,30000,5,471,디지털기기,\n갤럭시워치액티브2 40미리 검정색 판매합니다\n사용감 많구요 작동 잘합니다\n배...,경남,0.125000,0,0,144.8,1.061571,bluetooth,general,aluminium,갤럭시워치,44mm,active2
10329,갤럭시 워치 액티브2 40mm 팝니다,50000,2,59,디지털기기,\n액정에 기스 있어서 싸게 팝니다\n사용하던줄 같이 드려요\n동인천역 직거래 합니다\n,인천,0.125000,0,0,19.1,3.389831,bluetooth,general,aluminium,갤럭시워치,44mm,active2
10414,갤럭시워치 액티브2 40mm,60000,7,153,디지털기기,\n알루미늄 40mm모델로 대략 1년 정도 사용했으며 기변하여 내놓습니다. \n액정...,서울,0.166667,0,0,50.8,4.575163,bluetooth,general,aluminium,갤럭시워치,44mm,active2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14696,갤럭시워치 5 골프에디션 40mm 팝니다 (풀커버 스타일링팩 포함),330000,9,584,디지털기기,\n갤럭시워치 5 골프에디션 40mm 팝니다\n삼성 정품 풀커버 스타일링팩도 같이 ...,경기도,2.000000,0,0,181.5,1.541096,bluetooth,golf,aluminium,갤럭시워치,40mm,5
15225,(미개봉) 갤럭시워치5 골프에디션 40mm,320000,3,61,디지털기기,\n삼성 갤럭시워치5 골프에디션 40mm 입니다\n보시다시피 뜯지 않은 미개봉 품 ...,경기도,2.000000,0,1,20.4,4.918033,bluetooth,golf,aluminium,갤럭시워치,40mm,5
16246,갤럭시워치5 골프에디션 40mm,240000,1,217,디지털기기,\n갤럭시워치5 골프에디션 40mm 입니다. 8월31일 오후에 배송받고 하루 정도 ...,충북,3.000000,0,0,65.8,0.460829,bluetooth,golf,aluminium,갤럭시워치,40mm,5
18119,갤럭시워치5 골프에디션 팝니다(40mm),280000,7,471,디지털기기,\n갤럭시워치5 골프에디션 팝니다(40mm)\n미개봉 상품입니다.\n,서울,4.000000,0,1,146.2,1.486200,bluetooth,golf,aluminium,갤럭시워치,40mm,5


In [ ]:
df_apple_new.shape, df_galaxy_new.shape

((7313, 18), (5533, 18))

In [ ]:
df_galaxy_new.head(3)

,title,price,like,view,category,article,location,time,web_types,unused,attention,like/views(%),gps/cell,edition,material,model,size,series
10180,갤럭시 워치 액티브2 40mm 정품,150000,0,197,디지털기기,\n2020년도 구매후 1년정도차고 1년동안 방치했습니다. 새로운 주인만나서 사용되...,경기도,0.083333,0,0,59.1,0.000000,bluetooth,general,aluminium,갤럭시워치,44mm,active2
10275,갤럭시워치 액티브 2 40mm,65000,5,407,디지털기기,\n갤럭시워치 바꾸면서 쓰던거 내놓아용ㅎㅎ초기화 시켜놓은 상태이고 정상작동 다 잘됩...,울산,0.125000,0,0,125.6,1.228501,bluetooth,general,aluminium,갤럭시워치,44mm,active2
10276,갤럭시워치액티브2 40mm 판매합니다,30000,5,471,디지털기기,\n갤럭시워치액티브2 40미리 검정색 판매합니다\n사용감 많구요 작동 잘합니다\n배...,경남,0.125000,0,0,144.8,1.061571,bluetooth,general,aluminium,갤럭시워치,44mm,active2


In [ ]:
df_apple_new = df_apple_new[['title', 'price', 'like', 'view', 'category', 'article', 'location',
       'time', 'web_types', 'unused', 'attention', 'like/views(%)', 'gps/cell',
       'edition', 'material', 'model', 'size','series']].copy(deep = True)
df_apple_new.head(3)

,title,price,like,view,category,article,location,time,web_types,unused,attention,like/views(%),gps/cell,edition,material,model,size,series
1228,에르메스 애플워치 7세대 41mm 스테인리스 스틸 골드 싱글투어,1230000,5,442,디지털기기,\n에르메스 애플워치 7세대 41mm 스테인리스 스틸 골드 싱글투어 판매합니다\n2...,서울,0.375000,0,0,136.1,1.131222,gps,에르메스,stainless,애플워치,41mm,7
449,애플워치7 41mm (실버) 스테인리스 팝니다.,650000,0,209,디지털기기,\n애플워치7 41mm (실버) 스테인리스 팝니다.\n충전기 및 풀박스입니다.\n스...,경북,0.125000,0,0,62.7,0.000000,gps,일반,stainless,애플워치,41mm,7
602,애플워치 7 스테인리스 41mm실버 팝니다,600000,14,1164,디지털기기,\n구매후 실착용 10번도 하지 않은 상태 좋은 애플워치 7 스테인리스 팝니다\n배...,충북,0.166667,0,0,359.0,1.202749,gps,일반,stainless,애플워치,41mm,7


In [ ]:
df_galaxy_new.columns

Index(['title', 'price', 'like', 'view', 'category', 'article', 'location',
       'time', 'web_types', 'unused', 'attention', 'like/views(%)', 'gps/cell',
       'edition', 'material', 'model', 'size', 'series'],
      dtype='object')

In [ ]:
df = pd.concat([df_apple_new, df_galaxy_new])
df.columns 

Index(['title', 'price', 'like', 'view', 'category', 'article', 'location',
       'time', 'web_types', 'unused', 'attention', 'like/views(%)', 'gps/cell',
       'edition', 'material', 'model', 'size', 'series'],
      dtype='object')

In [ ]:
df = df[['title', 'price', 'like', 'view', 'category', 'article', 'location',
       'time', 'web_types', 'unused', 'like/views(%)', 'gps/cell',
       'edition', 'material', 'model', 'size', 'series', 'attention']].copy(deep = True)

In [ ]:
df

,title,price,like,view,category,article,location,time,web_types,unused,like/views(%),gps/cell,edition,material,model,size,series,attention
1228,에르메스 애플워치 7세대 41mm 스테인리스 스틸 골드 싱글투어,1230000,5,442,디지털기기,\n에르메스 애플워치 7세대 41mm 스테인리스 스틸 골드 싱글투어 판매합니다\n2...,서울,0.375000,0,0,1.131222,gps,에르메스,stainless,애플워치,41mm,7,136.1
449,애플워치7 41mm (실버) 스테인리스 팝니다.,650000,0,209,디지털기기,\n애플워치7 41mm (실버) 스테인리스 팝니다.\n충전기 및 풀박스입니다.\n스...,경북,0.125000,0,0,0.000000,gps,일반,stainless,애플워치,41mm,7,62.7
602,애플워치 7 스테인리스 41mm실버 팝니다,600000,14,1164,디지털기기,\n구매후 실착용 10번도 하지 않은 상태 좋은 애플워치 7 스테인리스 팝니다\n배...,충북,0.166667,0,0,1.202749,gps,일반,stainless,애플워치,41mm,7,359.0
1792,애플워치7 41mm 스테인리스 미사용팝니다,650000,3,118,디지털기기,\n처음에 한번켜보고 초기셋팅후 바로 박스에 보관되었던 제품입니다\n본체만입니다 밀...,서울,0.541667,0,0,2.542373,gps,일반,stainless,애플워치,41mm,7,37.5
1804,애플워치7 41mm 스테인리스 실버 미사용팝니다,650000,3,123,디지털기기,\n본체만 있습니다 밀레니즈루프줄은 팔리고 없습니다\n스타라이트 스포츠밴드 하나 가...,서울,0.541667,0,0,2.439024,gps,일반,stainless,애플워치,41mm,7,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14696,갤럭시워치 5 골프에디션 40mm 팝니다 (풀커버 스타일링팩 포함),330000,9,584,디지털기기,\n갤럭시워치 5 골프에디션 40mm 팝니다\n삼성 정품 풀커버 스타일링팩도 같이 ...,경기도,2.000000,0,0,1.541096,bluetooth,golf,aluminium,갤럭시워치,40mm,5,181.5
15225,(미개봉) 갤럭시워치5 골프에디션 40mm,320000,3,61,디지털기기,\n삼성 갤럭시워치5 골프에디션 40mm 입니다\n보시다시피 뜯지 않은 미개봉 품 ...,경기도,2.000000,0,1,4.918033,bluetooth,golf,aluminium,갤럭시워치,40mm,5,20.4
16246,갤럭시워치5 골프에디션 40mm,240000,1,217,디지털기기,\n갤럭시워치5 골프에디션 40mm 입니다. 8월31일 오후에 배송받고 하루 정도 ...,충북,3.000000,0,0,0.460829,bluetooth,golf,aluminium,갤럭시워치,40mm,5,65.8
18119,갤럭시워치5 골프에디션 팝니다(40mm),280000,7,471,디지털기기,\n갤럭시워치5 골프에디션 팝니다(40mm)\n미개봉 상품입니다.\n,서울,4.000000,0,1,1.486200,bluetooth,golf,aluminium,갤럭시워치,40mm,5,146.2


In [ ]:
df.head(3)

,title,price,like,view,category,article,location,time,web_types,unused,like/views(%),gps/cell,edition,material,model,size,series,attention
1228,에르메스 애플워치 7세대 41mm 스테인리스 스틸 골드 싱글투어,1230000,5,442,디지털기기,\n에르메스 애플워치 7세대 41mm 스테인리스 스틸 골드 싱글투어 판매합니다\n2...,서울,0.375000,0,0,1.131222,gps,에르메스,stainless,애플워치,41mm,7,136.1
449,애플워치7 41mm (실버) 스테인리스 팝니다.,650000,0,209,디지털기기,\n애플워치7 41mm (실버) 스테인리스 팝니다.\n충전기 및 풀박스입니다.\n스...,경북,0.125000,0,0,0.000000,gps,일반,stainless,애플워치,41mm,7,62.7
602,애플워치 7 스테인리스 41mm실버 팝니다,600000,14,1164,디지털기기,\n구매후 실착용 10번도 하지 않은 상태 좋은 애플워치 7 스테인리스 팝니다\n배...,충북,0.166667,0,0,1.202749,gps,일반,stainless,애플워치,41mm,7,359.0


In [ ]:
'''for 돌려서 
모델 애플워치 
'''
# model_list = df.model.unique().tolist().sort()
# series_list = df.series.unique().tolist().sort()
# edition_list = df.edition.unique().tolist().sort()
# size_list = df['size'].unique().tolist().sort()
# material_list = df['material'].unique().tolist().sort()
# cell_list = df['gps/cell'].unique().tolist().sort()

# for p in model_list:
#     for t in series_list:
#         for r in edition_list:
#             for k in size_list:
#                 for j in material_list:
#                     for i in cell_list:







'for 돌려서 \n모델 애플워치 \n'

In [ ]:
# import numpy as np
# df[''] = 
#상품 품질

In [ ]:
df
# 중고 평균시세로 100분위에서 
# scatter plot
# 분/시간/일 --> 일 통일
# 하루전 
# 0~1 / 1~2 /........./6~7
# 네 

,title,price,like,view,category,article,location,time,web_types,unused,like/views(%),gps/cell,edition,material,model,size,series,attention
1228,에르메스 애플워치 7세대 41mm 스테인리스 스틸 골드 싱글투어,1230000,5,442,디지털기기,\n에르메스 애플워치 7세대 41mm 스테인리스 스틸 골드 싱글투어 판매합니다\n2...,서울,0.375000,0,0,1.131222,gps,에르메스,stainless,애플워치,41mm,7,136.1
449,애플워치7 41mm (실버) 스테인리스 팝니다.,650000,0,209,디지털기기,\n애플워치7 41mm (실버) 스테인리스 팝니다.\n충전기 및 풀박스입니다.\n스...,경북,0.125000,0,0,0.000000,gps,일반,stainless,애플워치,41mm,7,62.7
602,애플워치 7 스테인리스 41mm실버 팝니다,600000,14,1164,디지털기기,\n구매후 실착용 10번도 하지 않은 상태 좋은 애플워치 7 스테인리스 팝니다\n배...,충북,0.166667,0,0,1.202749,gps,일반,stainless,애플워치,41mm,7,359.0
1792,애플워치7 41mm 스테인리스 미사용팝니다,650000,3,118,디지털기기,\n처음에 한번켜보고 초기셋팅후 바로 박스에 보관되었던 제품입니다\n본체만입니다 밀...,서울,0.541667,0,0,2.542373,gps,일반,stainless,애플워치,41mm,7,37.5
1804,애플워치7 41mm 스테인리스 실버 미사용팝니다,650000,3,123,디지털기기,\n본체만 있습니다 밀레니즈루프줄은 팔리고 없습니다\n스타라이트 스포츠밴드 하나 가...,서울,0.541667,0,0,2.439024,gps,일반,stainless,애플워치,41mm,7,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14696,갤럭시워치 5 골프에디션 40mm 팝니다 (풀커버 스타일링팩 포함),330000,9,584,디지털기기,\n갤럭시워치 5 골프에디션 40mm 팝니다\n삼성 정품 풀커버 스타일링팩도 같이 ...,경기도,2.000000,0,0,1.541096,bluetooth,golf,aluminium,갤럭시워치,40mm,5,181.5
15225,(미개봉) 갤럭시워치5 골프에디션 40mm,320000,3,61,디지털기기,\n삼성 갤럭시워치5 골프에디션 40mm 입니다\n보시다시피 뜯지 않은 미개봉 품 ...,경기도,2.000000,0,1,4.918033,bluetooth,golf,aluminium,갤럭시워치,40mm,5,20.4
16246,갤럭시워치5 골프에디션 40mm,240000,1,217,디지털기기,\n갤럭시워치5 골프에디션 40mm 입니다. 8월31일 오후에 배송받고 하루 정도 ...,충북,3.000000,0,0,0.460829,bluetooth,golf,aluminium,갤럭시워치,40mm,5,65.8
18119,갤럭시워치5 골프에디션 팝니다(40mm),280000,7,471,디지털기기,\n갤럭시워치5 골프에디션 팝니다(40mm)\n미개봉 상품입니다.\n,서울,4.000000,0,1,1.486200,bluetooth,golf,aluminium,갤럭시워치,40mm,5,146.2


In [ ]:
len(df.groupby(['model', 'series', 'edition', 'size', 'material', 'gps/cell']).price.mean().index)

125

In [ ]:
group = df.groupby(['model', 'series', 'edition', 'size', 'material', 'gps/cell']).price.mean().index
group

df.groupby(['model', 'series', 'edition', 'size', 'material', 'gps/cell']).price.mean()

dic = {}

for i in range(len(df.groupby(['model', 'series', 'edition', 'size', 'material', 'gps/cell']).price.mean().index)):
    dic[df.groupby(['model', 'series', 'edition', 'size', 'material', 'gps/cell']).price.mean().index[i]] =  df.groupby(['model', 'series', 'edition', 'size', 'material', 'gps/cell']).price.mean()[i]



In [ ]:
df_temp = df[['model', 'series', 'edition', 'size', 'material', 'gps/cell']]
df_temp.iloc[0].tolist()

['애플워치', '7', '에르메스', '41mm', 'stainless', 'gps']

In [ ]:
results = []
for i in range(len(df)):
    result = dic[tuple(df_temp.iloc[i].tolist())] 
    results.append(result)


In [ ]:
df.groupby(['model', 'series', 'edition', 'size', 'material', 'gps/cell']).price.mean().max()

model  series  edition  size  material   gps/cell 
갤럭시워치  1       general  42mm  aluminium  bluetooth     55555.555556
                                         lte           65000.000000
                        46mm  aluminium  bluetooth     56869.565217
                                         lte           86166.666667
       3       general  41mm  aluminium  bluetooth     99905.944700
                                                          ...      
애플워치   se      일반       40mm  aluminum   cellular     271345.454545
                                         gps          248100.450456
                        44mm  aluminum   cellular     301703.571429
                                         gps          450559.398374
                              stainless  gps          320000.000000
Name: price, Length: 125, dtype: float64

In [ ]:
df.tail(3)

,title,price,like,view,category,article,location,time,web_types,unused,like/views(%),gps/cell,edition,material,model,size,series,attention
16246,갤럭시워치5 골프에디션 40mm,240000,1,217,디지털기기,\n갤럭시워치5 골프에디션 40mm 입니다. 8월31일 오후에 배송받고 하루 정도 ...,충북,3.0,0,0,0.460829,bluetooth,golf,aluminium,갤럭시워치,40mm,5,65.8
18119,갤럭시워치5 골프에디션 팝니다(40mm),280000,7,471,디지털기기,\n갤럭시워치5 골프에디션 팝니다(40mm)\n미개봉 상품입니다.\n,서울,4.0,0,1,1.486200,bluetooth,golf,aluminium,갤럭시워치,40mm,5,146.2
7288,(미개봉) 갤럭시워치5 골프에디션 40mm,320000,3,179,웨어러블(워치),삼성 갤럭시워치5 골프에디션 40mm 입니다\n보시다시피 뜯지 않은 미개봉 품 이구...,서울,NaN,1,1,1.675978,bluetooth,golf,aluminium,갤럭시워치,40mm,5,55.8


In [ ]:
# 'model', 'series', 'edition', 'size', 'material', 'gps/cell'
dic[('애플워치', '7', '에르메스', '41mm', 'stainless', 'gps')]

1230000.0

In [ ]:
dic[('갤럭시워치', '5', 'golf', '40mm', 'aluminium', 'bluetooth')]

297857.14285714284

In [ ]:
df['average_price'] = results

In [ ]:
def r(x):
  return round(x)

df.average = df.average_price.copy(deep = True).apply(r)
df.average

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  after removing the cwd from sys.path.


1228     1230000
449       656429
602       656429
1792      656429
1804      656429
          ...   
14696     297857
15225     297857
16246     297857
18119     297857
7288      297857
Name: average_price, Length: 12846, dtype: int64

In [ ]:
df.head(3)

,title,price,like,view,category,article,location,time,web_types,unused,like/views(%),gps/cell,edition,material,model,size,series,attention,average_price
1228,에르메스 애플워치 7세대 41mm 스테인리스 스틸 골드 싱글투어,1230000,5,442,디지털기기,\n에르메스 애플워치 7세대 41mm 스테인리스 스틸 골드 싱글투어 판매합니다\n2...,서울,0.375000,0,0,1.131222,gps,에르메스,stainless,애플워치,41mm,7,136.1,1.230000e+06
449,애플워치7 41mm (실버) 스테인리스 팝니다.,650000,0,209,디지털기기,\n애플워치7 41mm (실버) 스테인리스 팝니다.\n충전기 및 풀박스입니다.\n스...,경북,0.125000,0,0,0.000000,gps,일반,stainless,애플워치,41mm,7,62.7,6.564286e+05
602,애플워치 7 스테인리스 41mm실버 팝니다,600000,14,1164,디지털기기,\n구매후 실착용 10번도 하지 않은 상태 좋은 애플워치 7 스테인리스 팝니다\n배...,충북,0.166667,0,0,1.202749,gps,일반,stainless,애플워치,41mm,7,359.0,6.564286e+05


In [ ]:
df.groupby(['model', 'series', 'edition', 'size', 'material', 'gps/cell']).price.mean().index

MultiIndex([('갤럭시워치',  '1', 'general', '42mm', 'aluminium', 'bluetooth'),
            ('갤럭시워치',  '1', 'general', '42mm', 'aluminium',       'lte'),
            ('갤럭시워치',  '1', 'general', '46mm', 'aluminium', 'bluetooth'),
            ('갤럭시워치',  '1', 'general', '46mm', 'aluminium',       'lte'),
            ('갤럭시워치',  '3', 'general', '41mm', 'aluminium', 'bluetooth'),
            ('갤럭시워치',  '3', 'general', '41mm', 'aluminium',       'lte'),
            ('갤럭시워치',  '3', 'general', '41mm', 'stainless', 'bluetooth'),
            ('갤럭시워치',  '3', 'general', '45mm', 'aluminium', 'bluetooth'),
            ('갤럭시워치',  '3', 'general', '45mm', 'aluminium',       'lte'),
            ('갤럭시워치',  '3', 'general', '45mm', 'stainless', 'bluetooth'),
            ...
            ( '애플워치',  '7',      '일반', '45mm',  'aluminum',       'gps'),
            ( '애플워치',  '7',      '일반', '45mm', 'stainless',  'cellular'),
            ( '애플워치',  '7',      '일반', '45mm', 'stainless',       'gps'),
            ( '애플워치', 

In [ ]:
df.model.unique().tolist()

['애플워치', '갤럭시워치']

In [ ]:
series_list = df.series.unique().tolist()
series_list.sort()

In [ ]:
df.edition.unique().tolist()

['에르메스', '일반', 'general', 'golf']

In [ ]:
df['size'].unique().tolist()

['41mm', '45mm', '40mm', '44mm', '38mm', '42mm', '46mm']

In [ ]:
df['size'].unique()

array(['41mm', '45mm', '40mm', '44mm', '38mm', '42mm', '46mm'],
      dtype=object)

In [ ]:
df['material'].unique().tolist()

['stainless', 'titanium', 'aluminum', 'aluminium', 'titanum']

In [ ]:
df['gps/cell'].unique().tolist()

['gps', 'cellular', 'bluetooth', 'lte']

In [ ]:
df.groupby(['model', 'series', 'edition', 'size', 'material', 'gps/cell']).price.mean()

model  series  edition  size  material   gps/cell 
갤럭시워치  1       general  42mm  aluminium  bluetooth     55555.555556
                                         lte           65000.000000
                        46mm  aluminium  bluetooth     56869.565217
                                         lte           86166.666667
       3       general  41mm  aluminium  bluetooth     99905.944700
                                                          ...      
애플워치   se      일반       40mm  aluminum   cellular     271345.454545
                                         gps          248100.450456
                        44mm  aluminum   cellular     301703.571429
                                         gps          450559.398374
                              stainless  gps          320000.000000
Name: price, Length: 125, dtype: float64

In [ ]:
'''
애플
41mm 45mm
시리즈별


---------------------
알루미늄 (스테인리스 티타늄)

일반 (에르메스)

GPS (셀룰러)

title -> model, 사이즈, 재질, 셀룰러 여부 ox
갤럭시

더 복잡...

--------------
상품 상태 기준정하기


미개봉/새상품,거의 --> 상
              --> 중
              --> 하
'''

'\n애플\n41mm 45mm\n시리즈별\n\n\n---------------------\n알루미늄 (스테인리스 티타늄)\n\n일반 (에르메스)\n\nGPS (셀룰러)\n\ntitle -> model, 사이즈, 재질, 셀룰러 여부 ox\n갤럭시\n\n더 복잡...\n\n--------------\n상품 상태 기준정하기\n\n\n미개봉/새상품,거의 --> 상\n              --> 중\n              --> 하\n'

In [ ]:
df.to_csv('df.csv')

In [ ]:
df.sample(5)

,title,price,like,view,category,article,location,time,web_types,unused,like/views(%),gps/cell,edition,material,model,size,series,attention,average_price
9297,애플워치 6 44미리 스테인레스 케어플러스 등록,440000,3,102,디지털기기,\n애플워치 44MM 스테인레스 스틸 그라파이트\n현재 애플케어플러스 가입 되어 있...,서울,4.000000,0,1,2.941176,gps,일반,stainless,애플워치,44mm,6,32.7,494607.142857
2523,애플워치6 스그 44m 나이키,510000,20,4809,웨어러블(워치),바로 쿨거 가능해요 배송 바로 해드려요\n\n🔥애플워치6 스그 44m 나이키🔥\n ...,경기도,NaN,1,0,0.415887,gps,일반,aluminum,애플워치,44mm,6,1456.7,308372.987138
559,애플워치7 41mm 그린알루미늄 gps 배터리100 보증기간22년10월말,310000,4,75,웨어러블(워치),배터리100% 보증기간22년10월말까지남았어요\n액정에 사진처럼 자잘하게 기스있어서...,경기도,NaN,1,0,5.333333,gps,일반,aluminum,애플워치,41mm,7,25.3,382312.815476
12980,갤럭시 워치4 클래식 42mm,160000,7,269,디지털기기,\nR4개월전 구매하고 사용은 별로 안했습니다\n정품충전기 같이 드려요\n스트랩은 ...,서울,1.000000,0,0,2.602230,bluetooth,general,aluminium,갤럭시워치,42mm,4 classic,85.6,187415.384615
11150,갤럭시워치5프로(pro) 미개봉 티타늄블랙 판매합니다+스타일링팩,400000,5,147,디지털기기,\n미개봉이구요 선물할사람이 있었는데 없었어요\n,서울,0.458333,0,0,3.401361,bluetooth,general,titanum,갤럭시워치,45mm,5 pro,47.6,404891.719745


### 상품 품질 column 추가

- 상 : 미개봉/미사용 상품 (unused = 1), '거의' 단어, s급, 최상, 깔끔, 상태 좋, 굿, 상태좋, 성능 100, 깨끗, 배터리 100, A급, 효율 100, 거의 사용, 기스 없, 새거, 사용감없, 찍힘 없
- 중
- 하 : 기스 있, 사용감 있, 오래된, 찍힘 있, 기스 약간, 생활기스, 생활 기스, 흠집 잇, 흠집 있, 사용감은 좀, 기스 조금, 기스있, 기스 좀, 사용감 많, 스크래치있, 스크래치 있
- 상/중 분류가 애매....
    - 절대적인 기준이 필요 : 상품 판매자는 대부분 자기 상품을 하자 없는 제품이라고 소개하기 떄문
    - 그리고 아예 박살난 상품은 중고로 팔지도 않을 듯..?
    - 대부분 거래 매물은 사용감이 적당히 있고 상태나 성능이 좋다고 설명
    - 실사용을 했다면(미개봉품이 아니라면) 이미 감가상각이 적용되어 "상"으로 분류X
**따라서 기본 가정으로 "대부분의 매물은 상태가 좋다"라고 설정하고 이를 중으로 분류**
**미개봉/새제품 (unused==1) : 상**
**흠집, 기스, 사용감 ⬆, 스크래치 : 하 **

In [ ]:
df_a = df[df["article"].str.contains("기스 있|사용감 있|사용감있|사용감이 조금|사용감은 있|찍힘 있|기스 약간|생활기스|흠집 잇|흠집 있|사용감은 좀|기스 조금|기스있|기스 좀|사용감 많|스크래치있|스크래치 있|스크래치가 조금|기스가 많|B급|유리 금|리퍼|기스 존재|깨졌|기스 진짜 조금|찍힘있|찍힘 조금|잔기스는 조금|긁혔|액정만 깨졌|기스는 당연히 있다만")== False].copy(deep=True) 
df_a["quality"] = "중"
df_a['quality'] = df_a['unused'].copy(deep=True).apply(lambda x : '상' if x==True else '중')
df_b = df[df["article"].str.contains("기스 있|사용감 있|사용감있|사용감이 조금|사용감은 있|찍힘 있|기스 약간|생활기스|흠집 잇|흠집 있|사용감은 좀|기스 조금|기스있|기스 좀|사용감 많|스크래치있|스크래치 있|스크래치가 조금|기스가 많|B급|유리 금|리퍼|기스 존재|깨졌|기스 진짜 조금|찍힘있|찍힘 조금|잔기스는 조금|긁혔|액정만 깨졌|기스는 당연히 있다만")].copy(deep=True) 
df_b["quality"] = "하"
df_final = pd.concat([df_a, df_b])
df_final

,title,price,like,view,category,article,location,time,web_types,unused,like/views(%),gps/cell,edition,material,model,size,series,attention,average_price,quality
1228,에르메스 애플워치 7세대 41mm 스테인리스 스틸 골드 싱글투어,1230000,5,442,디지털기기,\n에르메스 애플워치 7세대 41mm 스테인리스 스틸 골드 싱글투어 판매합니다\n2...,서울,0.375000,0,0,1.131222,gps,에르메스,stainless,애플워치,41mm,7,136.1,1.230000e+06,중
602,애플워치 7 스테인리스 41mm실버 팝니다,600000,14,1164,디지털기기,\n구매후 실착용 10번도 하지 않은 상태 좋은 애플워치 7 스테인리스 팝니다\n배...,충북,0.166667,0,0,1.202749,gps,일반,stainless,애플워치,41mm,7,359.0,6.564286e+05,중
1792,애플워치7 41mm 스테인리스 미사용팝니다,650000,3,118,디지털기기,\n처음에 한번켜보고 초기셋팅후 바로 박스에 보관되었던 제품입니다\n본체만입니다 밀...,서울,0.541667,0,0,2.542373,gps,일반,stainless,애플워치,41mm,7,37.5,6.564286e+05,중
1804,애플워치7 41mm 스테인리스 실버 미사용팝니다,650000,3,123,디지털기기,\n본체만 있습니다 밀레니즈루프줄은 팔리고 없습니다\n스타라이트 스포츠밴드 하나 가...,서울,0.541667,0,0,2.439024,gps,일반,stainless,애플워치,41mm,7,39.0,6.564286e+05,중
2376,애플워치 7 스테인리스 스틸 41mm,700000,2,123,디지털기기,"\n애플워치 7 스테인리스 스틸 41mm gps+셀룰러 모델이고, 그래파이트 색상입...",충남,1.000000,0,0,1.626016,gps,일반,stainless,애플워치,41mm,7,38.3,6.564286e+05,중
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13312,갤럭시 워치4 40mm 골프에디션 팝니다.,160000,12,774,디지털기기,\n전반적인 상태 양호합니다. \n액정에는 흠 같은 것 없이 깨끗합니다. \n테두리...,대구,1.000000,0,0,1.550388,bluetooth,golf,aluminium,갤럭시워치,40mm,4,240.6,1.955319e+05,하
15255,갤럭시워치5pro 프로모션 등록조차 안한 새상품,420000,9,760,디지털기기,\n2022.8월 사전예약 당일\n프로모션 등록안해서 제품넘버 등록하면 악세사리 마...,서울,2.000000,0,0,1.184211,bluetooth,general,aluminium,갤럭시워치,45mm,5 pro,234.3,3.652853e+05,하
15977,갤럭시 워치5 프로 팝니다.,350000,2,393,디지털기기,\n구매 후 2틀 사무실에서 착용 했습니다.\n눈에 띄지 않는 잔기스 조금 있네요....,충남,3.000000,0,0,0.508906,bluetooth,general,aluminium,갤럭시워치,45mm,5 pro,119.3,3.652853e+05,하
10524,갤럭시워치5 프로 lte 자급제 팝니다,430000,5,451,디지털기기,\n8월30일 주문하고 9월1일에 배송받았습니다\n개통없이 블투연결 잠시 사용한게 ...,경남,0.208333,0,0,1.108647,lte,general,aluminium,갤럭시워치,45mm,5 pro,138.8,4.300000e+05,하


In [ ]:
# 매물 품질 현황
df_final['quality'].value_counts()

중    8428
하    2576
상    1842
Name: quality, dtype: int64

In [ ]:
# 컬럼 순서 변경 -> 타겟값인 attention이 제일 뒤로
df_final = df_final[['title', 'price', 'article', 'like', 'view', 'category', 'location', 'time', 'web_types', 'unused',	'model', 'series', 'size', 'gps/cell', 'edition', 'material', 'like/views(%)', 'quality', 'average_price', 'attention']]

In [ ]:
df_final.shape

(12846, 20)

In [ ]:
df_final.sample(5)

,title,price,article,like,view,category,location,time,web_types,unused,model,series,size,gps/cell,edition,material,like/views(%),quality,average_price,attention
18708,(미개봉 새제품)갤럭시 워치4 44mm 골프에디션 판매,200000,\n미개봉 새상품 판매합니다.\n블랙 색상입니다\n,28,2079,디지털기기,인천,5.0,0,1,갤럭시워치,5,44mm,bluetooth,golf,aluminium,1.346801,상,203571.662420,643.3
12897,"상태 좋은 갤럭시워치3 41mm 실버, 골드 2개 팝니다",120000,"\n상태 좋은 갤럭시워치3 41mm 실버, 골드 2개 팝니다\n실버\n박스 풀셋에 ...",4,138,디지털기기,서울,1.0,0,0,갤럭시워치,3,41mm,bluetooth,general,aluminium,2.898551,중,99905.944700,44.2
2575,애플x나이키 워치se 44mm 스그 ss급팝니다,300000,케이스항상끼구 다녓고 찍힘 없어요\n\n배터리 성능 96\n\n스트랩 여유분이랑 스...,2,47,웨어러블(워치),인천,NaN,1,0,애플워치,se,44mm,gps,일반,aluminum,4.255319,중,450559.398374,15.5
3867,애플워치7 45mm 미드나이트 gps,340000,\n2022년 5월 제조품이구요\n풀박스 +밀레니얼 루프+ 케이스 입니다\n액정에 ...,3,152,디지털기기,대전,1.0,0,0,애플워치,7,45mm,gps,일반,aluminum,1.973684,하,417091.244866,47.7
18882,갤럭시워치 3 45mm 팔아요!,120000,\n아이폰으로 갈아타서 갤럭시 워치 3 45mm 팔아요!\n⭐️애플워치와 교환 가능...,11,280,디지털기기,경기도,5.0,0,0,갤럭시워치,3,45mm,bluetooth,general,aluminium,3.928571,중,106355.273438,91.7


In [ ]:
# 최종 전처리 완료된 데이터셋 저장
df_final.to_csv('df_real_final.csv')